In [1]:
pip install xlrd


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
def load_data(path):
    """Load the Super Store dataset from the given path and add new columns."""
    df = pd.read_excel(path)

    
    df['Shipping Days'] = (df['Ship Date'] - df['Order Date']).dt.days

    
    df['Profit per Sale'] = df.apply(lambda row: row['Profit'] / row['Sales'] if row['Sales'] != 0 else 0, axis=1)

    
    df['Discount per Quantity'] = df.apply(lambda row: row['Discount'] / row['Quantity'] if row['Quantity'] != 0 else 0, axis=1)

    return df

In [4]:
def preprocess_data(df, features):
    X = df[features].copy()
    X = X.dropna()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, X.index

def plot_elbow(X_scaled):
    inertia = []
    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X_scaled)
        inertia.append(kmeans.inertia_)
    plt.figure(figsize=(8, 4))
    plt.plot(range(2, 11), inertia, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.title('Elbow Method For Optimal k')
    plt.show()

def fit_kmeans(X_scaled, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    return labels, kmeans

def evaluate_clustering(X_scaled, labels):
    print('\n--- Clustering Validation Metrics ---')
    sil = silhouette_score(X_scaled, labels)
    ch = calinski_harabasz_score(X_scaled, labels)
    db = davies_bouldin_score(X_scaled, labels)
    print(f'Silhouette Score: {sil:.3f}')
    print(f'Calinski-Harabasz Index: {ch:.3f}')
    print(f'Davies-Bouldin Index: {db:.3f}')

def visualize_clusters(df, features, cluster_col):
    sns.pairplot(df, vars=features, hue=cluster_col, palette='tab10')
    plt.suptitle('Cluster Visualization', y=1.02)
    plt.show()

def cluster_profiling(df, features, cluster_col):
    for c in sorted(df[cluster_col].unique()):
        print(f'\nCluster {c}:')
        print(df[df[cluster_col] == c][features].describe())

In [5]:
def main():
    data_path = 'C:\Kmeans\Sample - Superstore (1).xls'  
    features = ['Sales', 'Quantity', 'Discount', 'Profit']

    df = load_data(data_path)
    X_scaled, valid_idx = preprocess_data(df, features)
    print(' Data loaded and new columns added.')

    print('\n--- Elbow Method ---')
    plot_elbow(X_scaled)

    n_clusters = 4 
    labels, kmeans = fit_kmeans(X_scaled, n_clusters)
    df.loc[valid_idx, 'Cluster'] = labels

    evaluate_clustering(X_scaled, labels)

    print('\n--- Cluster Means ---')
    print(df.groupby('Cluster')[features].mean())

    print('\n--- Cluster Visualization ---')
    visualize_clusters(df.loc[valid_idx], features, 'Cluster')

    print('\n--- Cluster Profiling ---')
    cluster_profiling(df.loc[valid_idx], features, 'Cluster')

  
    df.to_excel('output_superstore_with_new_columns.xlsx', index=False)
    print('\n Output saved as: output_superstore_with_new_columns.xlsx')

In [6]:
if __name__ == '_main_':
    main()